In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [7]:
# Model definition
class CIFAR10_MLP(nn.Module):
    def __init__(self, input_dim=3*32*32, hidden_dims=[2048, 1024, 512, 256], num_classes=10, dropout=0.5):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hdim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hdim))
            layers.append(nn.BatchNorm1d(hdim))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dropout))
            prev_dim = hdim
        layers.append(nn.Linear(prev_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten
        return self.net(x)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transforms: normalize CIFAR-10 images
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.247, 0.243, 0.261))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.247, 0.243, 0.261))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = CIFAR10_MLP().to(device)
model = torch.jit.script(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [5]:
from utils import train

train_metrics, val_metrics, test_metrics = train(model, train_loader, None, test_loader, 30, optimizer, criterion)

Epoch: 1 Total_Time: 2.2621 Average_Time_per_batch: 0.0058 Train_Accuracy: 0.2778 Train_Loss: 1.9698 
Epoch: 2 Total_Time: 1.7568 Average_Time_per_batch: 0.0045 Train_Accuracy: 0.3492 Train_Loss: 1.7906 
Epoch: 3 Total_Time: 1.8371 Average_Time_per_batch: 0.0047 Train_Accuracy: 0.3748 Train_Loss: 1.7267 
Epoch: 4 Total_Time: 1.7929 Average_Time_per_batch: 0.0046 Train_Accuracy: 0.3913 Train_Loss: 1.6829 
Epoch: 5 Total_Time: 2.0411 Average_Time_per_batch: 0.0052 Train_Accuracy: 0.4054 Train_Loss: 1.6491 
Epoch: 6 Total_Time: 1.9651 Average_Time_per_batch: 0.0050 Train_Accuracy: 0.4134 Train_Loss: 1.6267 
Epoch: 7 Total_Time: 1.8747 Average_Time_per_batch: 0.0048 Train_Accuracy: 0.4218 Train_Loss: 1.6057 
Epoch: 8 Total_Time: 1.9524 Average_Time_per_batch: 0.0050 Train_Accuracy: 0.4284 Train_Loss: 1.5874 
Epoch: 9 Total_Time: 1.3347 Average_Time_per_batch: 0.0034 Train_Accuracy: 0.4348 Train_Loss: 1.5715 
Epoch: 10 Total_Time: 1.7597 Average_Time_per_batch: 0.0045 Train_Accuracy: 0.4363

In [52]:
from spn_3.spn import SPN as SPN_3
    
model_3 = SPN_3(3*32*32, 100, 10, True).cuda()
#model_3.compile()

In [53]:
model_3 = torch.jit.trace(model_3, torch.randn(128, 3*32*32).cuda())
optimizer = optim.Adam(model_3.parameters())
criterion = nn.CrossEntropyLoss()

In [54]:
train_metrics_3, val_metrics_3, test_metrics_3 = train(model_3, train_loader, None, test_loader, 30, optimizer, criterion)

Epoch: 1 Total_Time: 0.9092 Average_Time_per_batch: 0.0023 Train_Accuracy: 0.3873 Train_Loss: 1.7884 
Epoch: 2 Total_Time: 0.7382 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.4543 Train_Loss: 1.5908 
Epoch: 3 Total_Time: 0.7353 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.4830 Train_Loss: 1.5121 
Epoch: 4 Total_Time: 0.6460 Average_Time_per_batch: 0.0017 Train_Accuracy: 0.5031 Train_Loss: 1.4537 
Epoch: 5 Total_Time: 0.7336 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.5209 Train_Loss: 1.4035 
Epoch: 6 Total_Time: 0.7680 Average_Time_per_batch: 0.0020 Train_Accuracy: 0.5334 Train_Loss: 1.3694 
Epoch: 7 Total_Time: 0.7701 Average_Time_per_batch: 0.0020 Train_Accuracy: 0.5495 Train_Loss: 1.3181 
Epoch: 8 Total_Time: 0.7242 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.5577 Train_Loss: 1.2977 
Epoch: 9 Total_Time: 0.6793 Average_Time_per_batch: 0.0017 Train_Accuracy: 0.5763 Train_Loss: 1.2417 
Epoch: 10 Total_Time: 0.7261 Average_Time_per_batch: 0.0019 Train_Accuracy: 0.5787

In [35]:
import torch
from torch import nn

hidden_dims = [2048, 1024, 512, 256]
total = sum(hidden_dims)

blocks = len(hidden_dims)
features = 3 * 32 * 32
neural_blocks = []
for dim in hidden_dims:
    std_dev = torch.sqrt(torch.tensor(1 / features)).to(device)
    neural_blocks.append(torch.randn(dim, features).to(device) * std_dev)
    features += dim

feature_blocks = []
features_start = 0
for i in range(len(neural_blocks)):
    features_end = neural_blocks[i].shape[1]
    block = neural_blocks[i][:, features_start:]
    for j in range(i + 1, len(neural_blocks)):
        block = torch.cat((block, neural_blocks[j][:, features_start:features_end]), dim=0)
    feature_blocks.append(nn.Parameter(block))
    features_start = features_end

biases = biases = nn.Parameter(torch.empty(total).uniform_(0.0, 1.0)).to(device)

In [36]:
from spn_2.spn import SPN as SPN_3
    
model_3 = SPN_3(3*32*32, 500, 10, False).cuda()
model_3.weights.extend(feature_blocks)
model_3.biases = biases

In [37]:
#model_3 = torch.jit.trace(model_3, torch.randn(128, 3*32*32).cuda())
optimizer = optim.Adam(model_3.parameters())
criterion = nn.CrossEntropyLoss()

In [38]:
from utils import train
train_metrics_3, val_metrics_3, test_metrics_3 = train(model_3, train_loader, None, test_loader, 50, optimizer, criterion)

Epoch: 1 Total_Time: 1.4126 Average_Time_per_batch: 0.0036 Train_Accuracy: 0.3786 Train_Loss: 2.1872 
Epoch: 2 Total_Time: 1.3031 Average_Time_per_batch: 0.0033 Train_Accuracy: 0.4477 Train_Loss: 1.6978 
Epoch: 3 Total_Time: 1.4158 Average_Time_per_batch: 0.0036 Train_Accuracy: 0.4779 Train_Loss: 1.6369 
Epoch: 4 Total_Time: 1.3471 Average_Time_per_batch: 0.0034 Train_Accuracy: 0.4992 Train_Loss: 1.6036 
Epoch: 5 Total_Time: 1.2950 Average_Time_per_batch: 0.0033 Train_Accuracy: 0.5262 Train_Loss: 1.5203 
Epoch: 6 Total_Time: 1.2255 Average_Time_per_batch: 0.0031 Train_Accuracy: 0.5466 Train_Loss: 1.4901 
Epoch: 7 Total_Time: 1.3340 Average_Time_per_batch: 0.0034 Train_Accuracy: 0.5669 Train_Loss: 1.4390 
Epoch: 8 Total_Time: 1.2986 Average_Time_per_batch: 0.0033 Train_Accuracy: 0.5885 Train_Loss: 1.3991 
Epoch: 9 Total_Time: 1.4968 Average_Time_per_batch: 0.0038 Train_Accuracy: 0.6074 Train_Loss: 1.3449 
Epoch: 10 Total_Time: 1.4345 Average_Time_per_batch: 0.0037 Train_Accuracy: 0.6280